<a href="https://colab.research.google.com/github/ajia90/smilestransformer/blob/main/interpolation_notgt_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports/ setup

mask + target = source -> molecules w/ some errors |||
mask + only start tokens -> only start toekn 


---



In [ ]:
# Install RDKit. Takes 2-3 minutes
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge python=3.7 
#!time conda install -q -y -c conda-forge rdkit 


--2021-06-02 02:05:08--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-06-02 02:05:08--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  89.87M   137MB/s    in 0.7s    

2021-06-02 02:05:09 (137 MB/s) - ‘Miniconda3-latest-Linux

In [ ]:
# Install RDKit. Takes 2-3 minutes
#%%capture
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge python=3.7
#!time conda install -q -y -c conda-forge rdkit 
!time conda install -q -y -c conda-forge rdkit rdkit=2020.09.2 #works

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

# training the model


In [ ]:
!conda install pytorch torchvision -c pytorch


Solving environment: \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    bzip2-1.0.8                |       h7b6447c_0          78 KB
    certifi-2021.5.30          |   py37h06a4308_0         139 KB
    conda-4.10.1               |   py37h06a4308_1         2.9 MB
    cudatoolkit-10.2.89        |       hfd86e86_1       365.1 MB
    ffmpeg-4.3                 |       hf484d3e_0         9.9 MB  pytorch
    freetype-2.10.4            |       h5ab3b9f_0         596 KB
    gmp-6.2.1                  |       h2531618_2         539 KB
    gnutls-3.6.15              |       he1e5248_0         1.0 MB
    intel-openmp-2021.2.0      |     h06a4308_610         1.3 MB

In [ ]:
#!conda install pytorch torchvision -c pytorch

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
 !python pretrain_trfm_target_zero.py

# methods/model


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import DrawingOptions
from torch.nn import functional as F


In [ ]:
def plot_mols(mols, unit=200, w=120, h=200, fontsize=1.0):
    drawer = Draw.MolDraw2DSVG(4*unit, 3*unit, w, h)

    # optをとり出しておく
    opt = drawer.drawOptions()
    opt.padding = 0.1
    opt.legendFontSize = 20
    #opt.atomfontSize = 20

    xs = np.array([0,1,2,3,0,1,2,3,0,1,2,3])*unit
    ys = np.array([0,0,0,0,1,1,1,1,2,2,2,2])*unit
    for i, (mol, x, y) in enumerate(zip(mols,xs,ys)):
        # SetOffsetで左上の座標を指定できる
        drawer.SetOffset(int(x), int(y))
        drawer.SetFontSize(fontsize)

        AllChem.Compute2DCoords(mol)
        Chem.Kekulize(mol)
        # 分子をSVGに書く
        drawer.DrawMolecule(mol, legend=str(i))


    # </svg> 書く
    drawer.FinishDrawing()
    return drawer

In [ ]:
def get_inputs(sm):
    #
    seq_len = 220
    sm = sm.split()
    if len(sm)>218:
        print('SMILES is too long ({:d})'.format(len(sm)))
        sm = sm[:109]+sm[-109:]
    ids = [vocab.stoi.get(token, unk_index) for token in sm]
    ids = [sos_index] + ids + [eos_index]
    seg = [1]*len(ids)
    padding = [pad_index]*(seq_len - len(ids))
    ids.extend(padding), seg.extend(padding)
    return ids, seg

def get_array(smiles):
    x_id, x_seg = [], []
    for sm in smiles:
        a,b = get_inputs(sm)
        x_id.append(a)
        x_seg.append(b)
    return torch.tensor(x_id), torch.tensor(x_seg)

In [ ]:
import torch
from pretrain_trfm_target_zero import TrfmSeq2seq
from build_vocab import WordVocab
from utils import split
from utils import validity

pad_index = 0
unk_index = 1
eos_index = 2
sos_index = 3
mask_index = 4

vocab = WordVocab.load_vocab('vocab.pkl')

# trfm_c= TrfmSeq2seq(len(vocab), 256, len(vocab), 4).cuda()
# trfm_c.load_state_dict(torch.load('trfm_new_12_80000.pkl'))
# trfm_c.eval()

trfm = TrfmSeq2seq(len(vocab), 256, len(vocab), 4)
trfm.load_state_dict(torch.load('trfm_notgt_12_90000.pkl'))
trfm.eval()
print('Total parameters:', sum(p.numel() for p in trfm.parameters()))

Total parameters: 4244013


In [ ]:
smiles_dict = vocab.stoi

In [ ]:
smiles_dict


# data


In [ ]:
#read in BBBp data
df = pd.read_csv('BBBP.csv')
print(df.shape)
df.head()

(2050, 4)


,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


In [ ]:
#sample of chembl25 data
df2 = pd.read_csv('smiles_sample2.csv')


(1052, 1)


In [ ]:
df.head()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


In [ ]:
x_split = [split(sm) for sm in df2['canonical_smiles'].values]
xid, xseg = get_array(x_split)

In [ ]:
xid.shape

torch.Size([1052, 220])

In [ ]:
smile_lengths = {}
#for i in range(df2.shape[0]):
for k in range(df2.shape[0]):
  sl1 = len(df2['canonical_smiles'][k])
  sl2 = len(df2['canonical_smiles'][0])
  if (sl1 == sl2):
    if sl1 in smile_lengths.keys():
        smile_lengths[sl1].append((k))
    else:
        smile_lengths[sl1] = [(k)]

In [ ]:
index = smile_lengths[27]
same_length_smiles = df2.loc[index]
same_length_smiles

,canonical_smiles
0,Cc1cc(cn1C)c2csc(N=C(N)N)n2
72,NC(=O)c1ccccc1n2cnc3ccccc23
241,COc1ccc2c(O)cc(OC)c(O)c2c1O
292,Nc1ccc2nc(oc2c1)c3ccc(F)cc3
302,CCOc1cc2c(cn1)[nH]c3ccccc23
304,O=C(c1ccccc1)c2nccc3ccccc23
312,CC(C)(C)C(N)C(=O)N1CCCC1C#N
445,[O-][S+]1N(Sc2ccccc12)C3CC3
547,COCC(C)(CS(=O)(=O)O)N(Cl)Cl
597,CC(C)(C)C(=O)Cn1c[n+](N)cn1


In [ ]:
x_split = [split(sm) for sm in same_length_smiles['canonical_smiles'].values]
xid, xseg = get_array(x_split)

# encode


In [ ]:
from dataset import Seq2seqDataset
from torch.utils.data import DataLoader
from tqdm import tqdm


In [ ]:
#encode function
X = trfm.encode(torch.t(xid))
print(X.shape)

(220, 15, 256)


In [ ]:
# #trfm.encoder
# dataset = Seq2seqDataset(df2['canonical_smiles'].values, vocab)
# data_loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=16)
# output_total = torch.empty(size=(xid.shape[1], xid.shape[0], 256))
# print(output_total.size)
# for b, sm in tqdm(enumerate(data_loader)):
#   # sm = torch.t(sm.cuda()) # (T,B)
#   # output1 = trfm_c(sm) # (T,,V)
#   embedded = trfm.embed(torch.t(sm.cuda()))  # (T,B,H)
#   embedded = trfm.pe(embedded) # (T,B,H)
#   output_total[:,b:b+4,:] = trfm.encoder(embedded)
# #output = output.detach().numpy()

# New Section

# mask


In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

def make_std_mask(tgt, pad):
    "Create a mask to hide padding and future words."
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(
        subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
    return tgt_mask

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


# decode

tgt = xid //source


In [ ]:
mask1 = generate_square_subsequent_mask(torch.t(xid).shape[0])

In [ ]:
mask1.shape

torch.Size([220, 220])

In [ ]:
# #trfm.decoder
# decoded = trfm.decoder(output, output, mask1)
# out = trfm.out(decoded) # (T,B,V)
# out = F.log_softmax(out, dim=2)
# out = out.detach().numpy()

In [ ]:
#decode function
hidden = torch.from_numpy(X).float()
decoded_output = trfm.decode(hidden)

In [ ]:
decoded.shape

(220, 15, 45)

# get smiles from decoded output


In [ ]:
#make compressed into sm
def get_smiles(decoded):
  _, next_word = torch.max(torch.from_numpy(decoded), dim = 2)
  decoded_smiles = torch.t(next_word).detach().numpy()
  #y =torch.t(xid).detach().numpy()
  #value -> smiles
  smiles_molecules = np.empty([decoded_smiles.shape[0],decoded_smiles.shape[1]], dtype=object)
  for i in range(decoded_smiles.shape[0]):   
    smiless = [list(smiles_dict.keys())[list(smiles_dict.values()).index(elem)] for elem in decoded_smiles[i]]
    smiles_molecules[i] = smiless
  #put all characters into one continuous string
  smiles_formatted = np.empty(decoded_smiles.shape[0], dtype=object)
  for i in range(smiles_molecules.shape[0]):
    smile = smiles_molecules[i]
    end = np.where(smile == '<eos>')[0][0]
    smiles_formatted[i] = "".join(smile[1:end])
  return smiles_formatted


chembl25 data

In [ ]:
molecules = get_smiles(decoded_output)

In [ ]:
same_length_smiles

,canonical_smiles
0,Cc1cc(cn1C)c2csc(N=C(N)N)n2
72,NC(=O)c1ccccc1n2cnc3ccccc23
241,COc1ccc2c(O)cc(OC)c(O)c2c1O
292,Nc1ccc2nc(oc2c1)c3ccc(F)cc3
302,CCOc1cc2c(cn1)[nH]c3ccccc23
304,O=C(c1ccccc1)c2nccc3ccccc23
312,CC(C)(C)C(N)C(=O)N1CCCC1C#N
445,[O-][S+]1N(Sc2ccccc12)C3CC3
547,COCC(C)(CS(=O)(=O)O)N(Cl)Cl
597,CC(C)(C)C(=O)Cn1c[n+](N)cn1


In [ ]:
same_length_smiles2

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
72,74,2,1,Cc1csc(n1)N=C(N)N
241,243,trifluperidol,1,OC1(CCN(CCCC(=O)c2ccc(F)cc2)CC1)c3cccc(c3)C(F)...
292,294,Mifepristone,0,CC#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CCC4=C3[...
302,304,mepiramine,0,COc1ccc(CN(CCN(C)C)c2ccccn2)cc1
304,306,mestranol,0,COc1ccc2[C@H]3CC[C@@]4(C)[C@@H](CC[C@@]4(O)C#C...
312,314,Mianserin,1,CN1CCN2C(C1)c3ccccc3Cc4ccccc24
445,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...
547,549,isoxicam,0,CN1C(=C(\O)Nc2cc(C)on2)/C(=O)c3ccccc3[S]1(=O)=O
597,599,salbutamol,0,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1


In [ ]:
index = smile_lengths[27]
same_length_smiles2 = df.loc[index]
x_split2 = [split(sm) for sm in same_length_smiles2['smiles'].values]
xid2, xseg2 = get_array(x_split2)
#encode function
X2 = trfm.encode(torch.t(xid2))
#decode function
hidden2 = torch.from_numpy(X2).float()
decoded_output2 = trfm.decode(hidden2)
molecules2 = get_smiles(decoded_output2)

In [ ]:
decoded_final = pd.DataFrame(molecules)
decoded_final.columns = ['smiles']
decoded_final

,smiles
0,Cc1cc(cn1C]c2csc(N=C(N]N]n2
1,NC(=O]c1ccccc1n2cnc3ccccc23
2,COc1ccc2c(O]cc(OC]c(O]c2c1O
3,Nc1ccc2nc(oc2c1]c3ccc(F]cc3
4,CCOc1cc2c(cn1][nH]c3ccccc23
5,O=C(c1ccccc1]c2nccc3ccccc23
6,CC(C](C]C(N]C(=O]N1CCCC1CsN
7,[O-][S+]1N(Sc2ccccc12]C3CC3
8,COCC(C](CS(=O](=O]O]N(Cl]Cl
9,CC(C](C]C(=O]Cn1c[n+](N]cn1


In [ ]:
pd.DataFrame(molecules2)

,0
0,[Cl].CC(C]NCC(O]COc1cccc2ccccc12
1,Cc1csc(n1]N=C(N]N
2,OC1(CCN(CCCC(=O]c2ccc(F]cc2]CC1]c3cccc(c3]C(F]...
3,CCsC[C@]1(O]CC[C@H]2[C@@H]3CCC4=CC(=O]CCC4=C3[...
4,COc1ccc(CN(CCN(C]C]c2ccccn2]cc1
5,COc1ccc2[C@H]3CC[C@@]4(C][C@@H](CC[C@@]4(O]CsC...
6,CN1CCN2C(C1]c3ccccc3Cc4ccccc24
7,CC1=CN([C@H]2C[C@H](N=[N+]=[N-]][C@@H](CO]O2]C...
8,CN1C(=C(\O]Nc2cc(C]on2]/C(=O]c3ccccc3[S]1(=O]=O
9,CC(C](C]NCC(O]c1ccc(O]c(CO]c1


# New Section

In [ ]:
def linear_interpolation(mol_from, mol_to, steps):
    n = steps + 1
    diff = mol_to - mol_from
    inter = mol_from + (1 / steps) * diff
    inter = np.reshape(inter, (220,1,256))
    for i in range(2,n):
      add = mol_from + (i / steps) * diff
      add = np.reshape(add, (220,1,256))
      inter = np.hstack((add, inter))
    return inter

In [ ]:
def decoded_interpolations(mol_from, mol_to, steps):
  molecule_morph = linear_interpolation(mol_from, mol_to, steps)
  decoded_interpolation = trfm.decode(torch.from_numpy(molecule_morph).float())
  interpolation_smiles = get_smiles(decoded_interpolation)
  interpolations = pd.DataFrame(interpolation_smiles)
  interpolations.columns = ['canonical_smiles']
  return interpolations

In [ ]:
mol1 = X[:,2,:]
mol2 = X[:,4,:]
interpolations1 = decoded_interpolations(mol1, mol2, 15)
interpolations2 = decoded_interpolations(X[:,5,:], X[:,4,:], 15)
interpolations3 = decoded_interpolations(X[:,10,:], X[:,3,:], 15)
interpolations4 = decoded_interpolations(X[:,7,:], X[:,4,:], 15)
interpolations5 = decoded_interpolations(X[:,6,:], X[:,13,:], 15)

In [ ]:
interpolations1

,canonical_smiles
0,CCOc1cc2c(cn1][nH]c3ccccc23
1,CCOc1cc2c(cn1][nH]c3ccccc23
2,CCOc1cc2c(cn1][nH]c3ccccc23
3,CCOc1cc2c(cn1][nH]c3ccccc23
4,CCOc1cc2c(cn1][nH]c3ccccc23
5,COOc1cc2c(On1][O]]c3Occcc23
6,COOcscc2c(O]s][O]]c3Occcc]O
7,COOcscc2c(O]s][O]]c(Occcc]O
8,COOsscc2c(O]sc(O]]c(Occ]c]O
9,COO1ccc2c(O]cc(O]]c(O]c]c]O


In [ ]:
interpolations2

,canonical_smiles
0,CCOc1cc2c(cn1][nH]c3ccccc23
1,CCOc1cc2c(cn1][nH]c3ccccc23
2,CCOc1cc2c(cn1][nH]c3ccccc23
3,CCOc1cc2c(cn1][nH]c3ccccc23
4,CCOc1cc2c(cs1][nH]c3ccccc23
5,O]Oc1cc]c(cs1][nH]c3ccccc23
6,OFO(1cc]c(cssc[n]]c3ccccc23
7,OFO(ccc]c(cssc[n]]c3ccccc23
8,O=O(ccc]c(cs]c[n]]c3ccccc23
9,O=O(c1ccc(c1]c[nc]c3ccccc23


In [ ]:
interpolations3

,canonical_smiles
0,Nc1ccc2nc(oc2c1]c3ccc(F]cc3
1,Nc1ccc2nc(oc2c1]c3ccc(F]cc3
2,Nc1ccc2nc(oc2c1]c3ccc(F]cc3
3,Nc1ccc2nc(oc2c1]c3ccc(F]cc3
4,Nc1ccc2nc(oc2c1]c3ccc(F]cc3
5,Nc1ccc2]c(oc2c1]c3ccc(F]cc3
6,Nc1cN(2]c(oc2N1]c3ccc(F]cc3
7,N]11N(]]](N]]N1]]3c3c(F]c13
8,N]11N(]]](N]]N]]]3c3c(Fcc13
9,N]s1N(]]]=N]]N]]]3c3c(Fcc13


In [ ]:
interpolations4

,canonical_smiles
0,CCOc1cc2c(cn1][nH]c3ccccc23
1,CCOc1cc2c(cn1][nH]c3ccccc23
2,CCOc1cc2c(cn1][nH]c3ccccc23
3,CCOc1cc]c(cn1][nH]c3ccccc23
4,CCO]1cc]c(cn1][nH]c3ccccc23
5,COO][Sc]c((Ss][nH]c3ccccc23
6,[OO][S+]c((Ss][n]]c3ccc3c]3
7,[OO][S+]c((Ss][n]]c3]c]3]]3
8,[OO][S+]sN(Sc][s]]c3]c]3]]3
9,[O-][S+]1N(Sc][cc]c3]]]3]]3


In [ ]:
interpolations5

,canonical_smiles
0,O=C(Nc1ccccc1]c2cnn3ccccc23
1,O=C(Nc1ccccc1]c2cnn3ccccc23
2,O=C(Nc1ccccc1]c2cnn3ccccc23
3,O=C(Nc1ccccc1]c2cnn3ccccc23
4,O=C(Nc1ccccc1]c2cnn3ccccc23
5,O=((Nc1ccccc1]cOcns3ccccc23
6,O=((N(1cc(cc1(cOcns3ccccc23
7,O=((N(1cc(Nc1(cOcNs3cccccsN
8,O=((N(sc](Ncs(=OcNs3]]]1]sN
9,O]((N(s]](N]s(=O]N1]]]]1]sN


In [ ]:
valid_molecules = np.zeros(len(interpolations))
for i in range(len(interpolations)):
  valid_molecules[i] = validity(interpolations.iloc[i])

# plot molecules

In [ ]:
from IPython.display import SVG


In [ ]:

mols = [Chem.MolFromSmiles(sm) for sm in interpolations['canonical_smiles'].values]
dr = plot_mols(mols, 250, 175, 250, 1.1)
with open('bbbp_mol.svg', 'w') as f:
    f.write(dr.GetDrawingText())
SVG(dr.GetDrawingText())
#Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(300,150))

ArgumentError: ignored

In [ ]:
dr